In [1]:
import pandas as pd
import os

In [2]:
#dump_file = "20230612WG6_Trial_Sample50-export-20230616-013559.csv"
dump_file = "20230622WG6_AllElectivesPart1-export-20230702-050135.csv"
#dump_file = "20230622WG6_AllElectivesPart2-export-20230702-050210.csv"
raw = pd.read_csv(os.path.join("coded",dump_file), dtype='string').fillna("")

In [3]:
meta_cols = []
coder_cols = []
renamer = {}
for col in raw.columns:
    shortened = col
    if col[0:3] == "[M]":
        shortened = col[4:-2]
        meta_cols.append(shortened)
    if col[0:3] == "[C]":
        shortened = col[3:]
        coder_cols.append(shortened)
    renamer[col] = shortened


# first two meta columns are elective code and institution, which we use as the index
# third meta column is title
index_cols = meta_cols[0:2]
keep_cols = index_cols + coder_cols
base = raw.rename(columns=renamer)[keep_cols]
base

,elective,institution,"Barker, Lecia","Bradley, Steven","Hooshangi, Sara","Kunkeler, Thom","Sibia, Naaz (Naaz Sibia)"
0,COMP3421,Durham_England,22; SEP,,,SEP; 22,
1,COMP3491,Durham_England,SEC,,,SEC; DM; 10,
2,COMP3507,Durham_England,AL,,,AL; 09,
3,COMP3567,Durham_England,GIT; SPD; 21,,,GIT; SPD; 21,
4,COMP3587,Durham_England,SEP,,,SEP; 17,
...,...,...,...,...,...,...,...
137,CS4254,VirginiaTech_USA,NC,,,AR,
138,CS4604,VirginiaTech_USA,DM,,,DM,
139,CS4704,VirginiaTech_USA,SE,,,SE,
140,CS4784,VirginiaTech_USA,HCI,,,HCI,


In [4]:
long1 = base.melt(index_cols)
long1

,elective,institution,variable,value
0,COMP3421,Durham_England,"Barker, Lecia",22; SEP
1,COMP3491,Durham_England,"Barker, Lecia",SEC
2,COMP3507,Durham_England,"Barker, Lecia",AL
3,COMP3567,Durham_England,"Barker, Lecia",GIT; SPD; 21
4,COMP3587,Durham_England,"Barker, Lecia",SEP
...,...,...,...,...
705,CS4254,VirginiaTech_USA,"Sibia, Naaz (Naaz Sibia)",
706,CS4604,VirginiaTech_USA,"Sibia, Naaz (Naaz Sibia)",
707,CS4704,VirginiaTech_USA,"Sibia, Naaz (Naaz Sibia)",
708,CS4784,VirginiaTech_USA,"Sibia, Naaz (Naaz Sibia)",


In [5]:
long2 = long1.join(long1['value'].str.split('; ', expand=True))
long2

,elective,institution,variable,value,0,1,2,3
0,COMP3421,Durham_England,"Barker, Lecia",22; SEP,22,SEP,None,None
1,COMP3491,Durham_England,"Barker, Lecia",SEC,SEC,None,None,None
2,COMP3507,Durham_England,"Barker, Lecia",AL,AL,None,None,None
3,COMP3567,Durham_England,"Barker, Lecia",GIT; SPD; 21,GIT,SPD,21,None
4,COMP3587,Durham_England,"Barker, Lecia",SEP,SEP,None,None,None
...,...,...,...,...,...,...,...,...
705,CS4254,VirginiaTech_USA,"Sibia, Naaz (Naaz Sibia)",,,None,None,None
706,CS4604,VirginiaTech_USA,"Sibia, Naaz (Naaz Sibia)",,,None,None,None
707,CS4704,VirginiaTech_USA,"Sibia, Naaz (Naaz Sibia)",,,None,None,None
708,CS4784,VirginiaTech_USA,"Sibia, Naaz (Naaz Sibia)",,,None,None,None


In [6]:
long3 = long2.drop('value', axis=1)
long3 = long3.rename(columns={'variable': 'coder'})
long3

,elective,institution,coder,0,1,2,3
0,COMP3421,Durham_England,"Barker, Lecia",22,SEP,None,None
1,COMP3491,Durham_England,"Barker, Lecia",SEC,None,None,None
2,COMP3507,Durham_England,"Barker, Lecia",AL,None,None,None
3,COMP3567,Durham_England,"Barker, Lecia",GIT,SPD,21,None
4,COMP3587,Durham_England,"Barker, Lecia",SEP,None,None,None
...,...,...,...,...,...,...,...
705,CS4254,VirginiaTech_USA,"Sibia, Naaz (Naaz Sibia)",,None,None,None
706,CS4604,VirginiaTech_USA,"Sibia, Naaz (Naaz Sibia)",,None,None,None
707,CS4704,VirginiaTech_USA,"Sibia, Naaz (Naaz Sibia)",,None,None,None
708,CS4784,VirginiaTech_USA,"Sibia, Naaz (Naaz Sibia)",,None,None,None


In [7]:
long4 = long3.melt(index_cols + ['coder'])
long4 = long4.rename(columns={'value': 'code'})[index_cols + ['coder','code']]
long4.fillna("",inplace = True)
long4 = long4[long4.code.str.len()>0]
long4

,elective,institution,coder,code
0,COMP3421,Durham_England,"Barker, Lecia",22
1,COMP3491,Durham_England,"Barker, Lecia",SEC
2,COMP3507,Durham_England,"Barker, Lecia",AL
3,COMP3567,Durham_England,"Barker, Lecia",GIT
4,COMP3587,Durham_England,"Barker, Lecia",SEP
...,...,...,...,...
1942,CYBR4893,Kennesaw_USA,"Kunkeler, Thom",10
2572,COMP4187,Durham_England,"Kunkeler, Thom",10
2601,COMPSCI4080,Glasgow_Scotland,"Kunkeler, Thom",15
2627,inf-TM,CAU_Germany,"Kunkeler, Thom",14


In [8]:
long5 = long4.groupby(index_cols + ['code']).count().unstack().fillna(0)
long5.columns = long5.columns.get_level_values(1)
long5.to_csv(os.path.join("coded","codes_" + dump_file))
long5

,code,02,03,04,07,09,10,14,15,16,17,...,MSF,NC,OS,PDC,SDF,SE,SEC,SEP,SF,SPD
elective,institution,,,,,,,,,,,,,,,,,,,,,
1DL251,Uppsala_Sweden,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1DL301,Uppsala_Sweden,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1DL321,Uppsala_Sweden,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1DL481,Uppsala_Sweden,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1DL550,Uppsala_Sweden,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
inf-CI,CAU_Germany,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
inf-EAeS,CAU_Germany,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
inf-EERP,CAU_Germany,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
